In [3]:
from math import gcd, log2, ceil
import numpy as np
from random import randint
import sys
sys.path.append('./GTC_tutorial/data')
from qiskit.algorithms import Shor
import cirq
import qsimcirq
import qiskit_to_cirq_conversion
import fractions

In [4]:
!pwd

/staff/GTC_tutorial


In [5]:
ngpus = 1
qsim_options = qsimcirq.QSimOptions(
        max_fused_gate_size = 2
        , cpu_threads = 1
        , gpu_mode = ngpus
        , use_sampler = True
        , disable_gpu = False
    )
qsim_simulator = qsimcirq.QSimSimulator(qsim_options)

In [6]:
def get_num_bits(N):
    return int(log2(N)+1)

In [7]:
def get_histogram(result, width, shots):
    if shots == 1:
        return {result.tobytes(): 1}

    # We rely on the PY37+ feature that the dict is ordered, and sort the
    # measurement outcome (significant ones come first)
    out = {}
    axis = None if result.ndim == 1 else 0
    keys, values = np.unique(result, axis=axis, return_counts=True)
    idx = np.argsort(values)[::-1]
    keys = keys[idx]
    values = values[idx]
    for i in range(keys.shape[0]):
        # numpy arrays are not hashable
        if result.ndim == 2:  # from qsimcirq
            k = ''.join(str(keys[i])[1:-1].split())
        else:
            assert False
        out[k] = values[i]

    # sanity check
    counts = 0
    for k, v in out.items():
        counts += v
    assert counts == shots, f"counts = {counts}, shots = {shots}"

    if shots/len(out) < 5.0:
        print(f"WARNING: too many ({len(out)}) unique bitstrings with limited shots ({shots}), "
              "statistics may not be accurate")

    return out


In [8]:
def get_factors(r, r_nbits, x, N):
    assert r_nbits > 0
    assert x > 0
    assert N > 0
    eigenphase = float(r) / 2**r_nbits
    f = fractions.Fraction.from_float(eigenphase).limit_denominator(N)    
    # If the numerator is zero, the order finder failed.
    if f.numerator == 0:
        return None
    
    # Else, return the denominator if it is valid.
    r = f.denominator
    if x**r % N != 1:
        return None
    return r

In [9]:
def find_period_quantum(x, N):
    """The quantum algorithm to find the period r of x^r (mod N)."""
    
    print("Running quantum algorithm...")
    shots   = 128
    nbits   = get_num_bits(N)
    
    qiskit_circuit              = Shor().construct_circuit(N, x)
    cirq_circuit, qubit_offsets = qiskit_to_cirq_conversion.convert(qiskit_circuit)

    print("Total number of qubits:", qubit_offsets['total'])
    cirq_circuit.append(cirq.measure_each(*cirq.LineQubit.range(qubit_offsets['up'], qubit_offsets['down'])))

    result = qsim_simulator.sample(cirq_circuit,repetitions=shots)
    measurement_results = result.to_numpy()
    print("Measurement results:")
    print(measurement_results)
    hist = get_histogram(measurement_results, qubit_offsets['down'], shots)
    print("Number of occurences:", hist)
    most_probable_bitpattern = max(hist, key=hist.get)
    result = int(most_probable_bitpattern[::-1],2)
    print("Most probable bitpattern:", most_probable_bitpattern, "=", result)
    r = get_factors(result, qubit_offsets['down'], x, N)
    print("Found period:", r)
    # #print the circuit 
    # print(cirq_circuit)
    # #print the circuit using qiskit 
    # print(qiskit_circuit.draw())
    # #as the circuit depth is big so maybe it won't work for large N
    # # print("The circuit:")
    # # print(cirq_circuit)
    return r

In [10]:
def find_period(a, N):
    r = find_period_quantum(a, N)
    return r

In [11]:
def shors_algorithm(N):
    print("Running Shor's algorithm quantum on N =", N)
    while(True):
        # 1. Select a random integer between 2 and N-1
        a = randint(2,N-1)
        
        # 2. See if it happens to factor N
        print("Trying a =", a)
        g = gcd(a,N)
        if g != 1:
            print("Found factor by chance:", g)
            return (g, N//g)
        
        # 3. Find the period a^r = 1 (mod N)
        r = find_period(a, N)
        if r != None and r % 2 != 1 and (r//2) % N != N-1:
            a1 = gcd(a ** (r//2) + 1, N)
            print("Found factor:", a1)
            return (a1, N//a1)
            print("The resultL:")
            print(r)
        print("retrying...")

In [12]:
my_integer = 3*5
shors_algorithm(my_integer)

Running Shor's algorithm quantum on N = 15
Trying a = 9
Found factor by chance: 3


(3, 5)

In [13]:
my_integer = 5*5
shors_algorithm(my_integer)

Running Shor's algorithm quantum on N = 25
Trying a = 6
Running quantum algorithm...
Total number of qubits: 22
Measurement results:
[[0 1 1 ... 0 0 1]
 [1 0 1 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 1 0 ... 0 1 1]
 [0 1 1 ... 0 0 1]
 [1 1 0 ... 0 1 1]]
Number of occurences: {'0000000000': 35, '1100110011': 30, '0110011001': 16, '1011001100': 13, '0101100110': 10, '1001100110': 6, '1110011001': 3, '1101100110': 3, '0001011001': 3, '1101001100': 2, '0100110011': 2, '1111001100': 1, '1010011001': 1, '1000110011': 1, '0011001100': 1, '0000010110': 1}
Most probable bitpattern: 0000000000 = 0
Found period: None
retrying...
Trying a = 5
Found factor by chance: 5


(5, 5)

In [14]:
my_integer = 5*7
shors_algorithm(my_integer)

Running Shor's algorithm quantum on N = 35
Trying a = 27
Running quantum algorithm...
Total number of qubits: 26
Measurement results:
[[0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]]
Number of occurences: {'000000000011': 37, '000000000001': 34, '000000000010': 32, '000000000000': 25}
Most probable bitpattern: 000000000011 = 3072
Found period: 4
Found factor: 5


(5, 7)

In [12]:
my_integer = 5*9
shors_algorithm(my_integer)

Running Shor's algorithm quantum on N = 45
Trying a = 19
Running quantum algorithm...
Total number of qubits: 26
Measurement results:
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
Number of occurences: {'000000000000': 72, '000000000001': 56}
Most probable bitpattern: 000000000000 = 0
Found period: None
retrying...
Trying a = 37
Running quantum algorithm...
Total number of qubits: 26
Measurement results:
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 1 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
Number of occurences: {'000000000011': 36, '000000000010': 33, '000000000000': 32, '000000000001': 27}
Most probable bitpattern: 000000000011 = 3072
Found period: 4
Found factor: 5


(5, 9)

In [13]:
my_integer = 7*9
shors_algorithm(my_integer)

Running Shor's algorithm quantum on N = 63
Trying a = 31
Running quantum algorithm...
Total number of qubits: 26
Measurement results:
[[1 0 1 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [1 1 0 ... 1 0 1]
 ...
 [0 1 1 ... 1 0 0]
 [1 1 1 ... 0 1 1]
 [1 0 1 ... 0 1 1]]
Number of occurences: {'110101010101': 23, '000000000000': 22, '000000000001': 20, '101010101010': 18, '110101010100': 17, '101010101011': 11, '010101010101': 4, '011010101010': 2, '001010101011': 2, '011010101011': 2, '111010101010': 1, '010101010100': 1, '001011101011': 1, '011101010100': 1, '001010101010': 1, '110100101011': 1, '111010101011': 1}
Most probable bitpattern: 110101010101 = 2731
Found period: None
retrying...
Trying a = 9
Found factor by chance: 9


(9, 7)

In [15]:
my_integer = 11*9
shors_algorithm(my_integer)

Running Shor's algorithm quantum on N = 99
Trying a = 17
Running quantum algorithm...
Total number of qubits: 30


# References

* P. Shor, 1997 , "Polynomial-time algorithms for prime factorization and discrete logarithms on a quantum computer", SIAM J. Comp., 26, pp. 1484-150
* S. Beauregard, 2003, "Circuit for Shor's algorithm using 2n+3 qubits.", Quantum Info. Comput. 3, 2 (March 2003), 175–185.